# Data Preparation for Retail/CPG Planning Analytics

This notebook generates and prepares realistic datasets for a **global retail/CPG company's planning value chain**. The datasets cover end-to-end planning processes from demand planning to supply planning.

**Planning Value Chain Covered:**
- Demand Planning
- Supply Planning
- Production Planning
- Distribution Planning

**Datasets Generated:**

| Dataset | Type | Planning Process | Notebook |
|---------|------|------------------|----------|
| `demand_forecast` | Time Series | Demand Planning | 04_time_series_forecasting |
| `price_elasticity` | Regression | Demand Planning | 02_regression |
| `promotion_lift` | Regression | Demand Planning | 02_regression |
| `supplier_delay_risk` | Binary Classification | Supply Planning | 01_classification |
| `supplier_lead_time` | Regression | Supply Planning | 02_regression |
| `material_shortage` | Multi-class Classification | Supply Planning | 01_classification |
| `labor_shortage` | Multi-class Classification | Production Planning | 01_classification |
| `yield_prediction` | Regression | Production Planning | 02_regression |
| `scrap_anomaly` | Anomaly Detection | Production Planning | 03_outlier_detection |
| `transportation_lead_time` | Regression | Distribution Planning | 02_regression |
| `otif_risk` | Multi-class Classification | Distribution Planning | 01_classification |

**Run this notebook once** to set up all the data before running the analytics notebooks.

## Compute Setup

We recommend running this notebook on **Serverless Compute** with the **Base Environment V4**.

To configure:
1. Click on the compute selector in the notebook toolbar
2. Select **Serverless**
3. Under Environment, choose **Base Environment V4**

Serverless compute provides fast startup times and automatic scaling, ideal for interactive notebook workflows.

## 1. Install Required Packages

In [ ]:
%pip install pandas numpy --quiet

In [ ]:
dbutils.library.restartPython()

## 2. Configuration

Define the catalog and schema where datasets will be stored.

In [ ]:
# Configure your catalog and schema
CATALOG = "tabpfn_databricks"
SCHEMA = "default"

# Create the catalog and schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

print(f"Using catalog: {CATALOG}")
print(f"Using schema: {SCHEMA}")

## 3. Import Data Generation Utilities

We use custom data generation functions that create realistic retail/CPG planning datasets.

In [ ]:
import numpy as np
import pandas as pd
import sys
import os

# Add the scripts directory to the path so we can import util.py
# In Databricks, you may need to adjust this path based on your repo structure
repo_root = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
scripts_path = os.path.join(repo_root, 'scripts')
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

# Import data generation functions
from util import (
    generate_supplier_delay_risk_data,
    generate_material_shortage_data,
    generate_labor_shortage_data,
    generate_price_elasticity_data,
    generate_promotion_lift_data,
    generate_supplier_lead_time_data,
    generate_transportation_lead_time_data,
    generate_yield_prediction_data,
    generate_scrap_anomaly_data,
    generate_aggregate_demand_forecast_data,
    generate_otif_risk_data
)

print("Data generation utilities imported successfully!")

---
# Demand Planning Datasets
---

## 4. Demand Forecast Dataset (Time Series)

**Use Case:** Forecast product demand by category and region

This dataset contains monthly demand data across multiple product categories and regions, supporting demand forecasting and inventory planning processes.

In [ ]:
# Generate demand forecast data
df_demand_forecast = generate_aggregate_demand_forecast_data(n_series=50, n_months=36, seed=42)

print(f"Demand Forecast Dataset")
print(f"Shape: {df_demand_forecast.shape}")
print(f"Number of time series: {df_demand_forecast['series_id'].nunique()}")
print(f"Time range: {df_demand_forecast['date'].min()} to {df_demand_forecast['date'].max()}")
print(f"\nCategory distribution:")
print(df_demand_forecast['category'].value_counts())

display(df_demand_forecast.head(10))

In [ ]:
spark.createDataFrame(df_demand_forecast).write.mode("overwrite").saveAsTable("demand_forecast")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.demand_forecast")

## 5. Price Elasticity Dataset (Regression)

**Use Case:** Understand how price changes affect demand

This dataset helps demand planners and revenue management teams predict how changes in pricing will impact unit sales across different products and market conditions.

In [ ]:
df_price_elasticity = generate_price_elasticity_data(n_samples=3000, seed=42)

print(f"Price Elasticity Dataset")
print(f"Shape: {df_price_elasticity.shape}")
print(f"\nTarget (price_elasticity) statistics:")
print(df_price_elasticity['price_elasticity'].describe())
print(f"\nNote: Elasticity values are typically negative (demand decreases as price increases)")

display(df_price_elasticity.head(10))

In [ ]:
spark.createDataFrame(df_price_elasticity).write.mode("overwrite").saveAsTable("price_elasticity")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.price_elasticity")

## 6. Promotion Lift Dataset (Regression)

**Use Case:** Predict the sales impact of planned promotions

This dataset supports trade promotion planning by predicting the incremental sales lift from different promotion types, depths, and marketing support.

In [ ]:
df_promotion_lift = generate_promotion_lift_data(n_samples=2500, seed=42)

print(f"Promotion Lift Dataset")
print(f"Shape: {df_promotion_lift.shape}")
print(f"\nTarget (promotion_lift_pct) statistics:")
print(df_promotion_lift['promotion_lift_pct'].describe())
print(f"\nPromotion type distribution:")
print(df_promotion_lift['promotion_type'].value_counts())

display(df_promotion_lift.head(10))

In [ ]:
spark.createDataFrame(df_promotion_lift).write.mode("overwrite").saveAsTable("promotion_lift")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.promotion_lift")

---
# Supply Planning Datasets
---

## 7. Supplier Delay Risk Dataset (Binary Classification)

**Use Case:** Predict which supplier deliveries are at risk of delay

This dataset helps supply planners identify high-risk orders and take proactive mitigation actions such as expediting, finding alternative suppliers, or adjusting production schedules.

In [ ]:
df_supplier_delay = generate_supplier_delay_risk_data(n_samples=2000, seed=42)

print(f"Supplier Delay Risk Dataset")
print(f"Shape: {df_supplier_delay.shape}")
print(f"\nTarget distribution (is_delayed):")
print(df_supplier_delay['is_delayed'].value_counts())
print(f"\nDelay rate: {df_supplier_delay['is_delayed'].mean():.1%}")

display(df_supplier_delay.head(10))

In [ ]:
spark.createDataFrame(df_supplier_delay).write.mode("overwrite").saveAsTable("supplier_delay_risk")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.supplier_delay_risk")

## 8. Supplier Lead Time Dataset (Regression)

**Use Case:** Predict actual supplier delivery lead times

This dataset helps supply planners predict actual delivery times (vs. contracted lead times) to improve planning accuracy and reduce stockouts.

In [ ]:
df_supplier_lead_time = generate_supplier_lead_time_data(n_samples=2000, seed=42)

print(f"Supplier Lead Time Dataset")
print(f"Shape: {df_supplier_lead_time.shape}")
print(f"\nTarget (actual_lead_time_days) statistics:")
print(df_supplier_lead_time['actual_lead_time_days'].describe())
print(f"\nSupplier region distribution:")
print(df_supplier_lead_time['supplier_region'].value_counts())

display(df_supplier_lead_time.head(10))

In [ ]:
spark.createDataFrame(df_supplier_lead_time).write.mode("overwrite").saveAsTable("supplier_lead_time")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.supplier_lead_time")

## 9. Material Shortage Dataset (Multi-class Classification)

**Use Case:** Predict which materials are at risk of shortage

This dataset supports material planners in prioritizing procurement actions based on shortage risk levels (No Risk, At Risk, Critical).

In [ ]:
df_material_shortage = generate_material_shortage_data(n_samples=1500, seed=42)

print(f"Material Shortage Dataset")
print(f"Shape: {df_material_shortage.shape}")
print(f"\nTarget distribution (shortage_risk):")
print("0 = No Risk, 1 = At Risk, 2 = Critical")
print(df_material_shortage['shortage_risk'].value_counts().sort_index())

display(df_material_shortage.head(10))

In [ ]:
spark.createDataFrame(df_material_shortage).write.mode("overwrite").saveAsTable("material_shortage")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.material_shortage")

---
# Production Planning Datasets
---

## 10. Labor Shortage Dataset (Multi-class Classification)

**Use Case:** Predict labor availability issues at facilities

This dataset helps production and HR planners anticipate workforce shortages and take proactive actions like overtime scheduling, temp staffing, or cross-training.

In [ ]:
df_labor_shortage = generate_labor_shortage_data(n_samples=1500, seed=42)

print(f"Labor Shortage Dataset")
print(f"Shape: {df_labor_shortage.shape}")
print(f"\nTarget distribution (labor_shortage_risk):")
print("0 = Adequate, 1 = At Risk, 2 = Critical")
print(df_labor_shortage['labor_shortage_risk'].value_counts().sort_index())
print(f"\nFacility type distribution:")
print(df_labor_shortage['facility_type'].value_counts())

display(df_labor_shortage.head(10))

In [ ]:
spark.createDataFrame(df_labor_shortage).write.mode("overwrite").saveAsTable("labor_shortage")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.labor_shortage")

## 11. Yield Prediction Dataset (Regression)

**Use Case:** Predict production yield for capacity planning

This dataset helps production planners predict output yield based on equipment, materials, and process parameters to optimize capacity planning.

In [ ]:
df_yield = generate_yield_prediction_data(n_samples=2000, seed=42)

print(f"Yield Prediction Dataset")
print(f"Shape: {df_yield.shape}")
print(f"\nTarget (yield_percentage) statistics:")
print(df_yield['yield_percentage'].describe())
print(f"\nProduction line distribution:")
print(df_yield['production_line'].value_counts())

display(df_yield.head(10))

In [ ]:
spark.createDataFrame(df_yield).write.mode("overwrite").saveAsTable("yield_prediction")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.yield_prediction")

## 12. Scrap Anomaly Dataset (Anomaly Detection)

**Use Case:** Detect unusual scrap/defect patterns

This dataset helps production managers identify abnormal production runs that may indicate equipment issues, material problems, or process deviations.

In [ ]:
df_scrap_anomaly, anomaly_labels = generate_scrap_anomaly_data(n_samples=1000, anomaly_rate=0.08, seed=42)
df_scrap_anomaly['is_anomaly'] = anomaly_labels

print(f"Scrap Anomaly Dataset")
print(f"Shape: {df_scrap_anomaly.shape}")
print(f"\nAnomaly distribution:")
print(f"Normal (0): {(anomaly_labels == 0).sum()}")
print(f"Anomaly (1): {(anomaly_labels == 1).sum()}")
print(f"Anomaly rate: {anomaly_labels.mean():.1%}")

display(df_scrap_anomaly.head(10))

In [ ]:
spark.createDataFrame(df_scrap_anomaly).write.mode("overwrite").saveAsTable("scrap_anomaly")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.scrap_anomaly")

---
# Distribution Planning Datasets
---

## 13. Transportation Lead Time Dataset (Regression)

**Use Case:** Predict transportation/delivery lead times

This dataset helps distribution planners predict actual transit times for shipments to improve delivery planning and customer service levels.

In [ ]:
df_transport_lead_time = generate_transportation_lead_time_data(n_samples=2000, seed=42)

print(f"Transportation Lead Time Dataset")
print(f"Shape: {df_transport_lead_time.shape}")
print(f"\nTarget (actual_transit_days) statistics:")
print(df_transport_lead_time['actual_transit_days'].describe())
print(f"\nShipment type distribution:")
print(df_transport_lead_time['shipment_type'].value_counts())

display(df_transport_lead_time.head(10))

In [ ]:
spark.createDataFrame(df_transport_lead_time).write.mode("overwrite").saveAsTable("transportation_lead_time")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.transportation_lead_time")

## 14. OTIF Risk Dataset (Multi-class Classification)

**Use Case:** Predict On-Time-In-Full (OTIF) delivery risk

This dataset helps distribution planners identify orders at risk of failing to be delivered on-time and in-full, enabling proactive intervention to improve customer service levels.

In [ ]:
df_otif_risk = generate_otif_risk_data(n_samples=2000, seed=42)

print(f"OTIF Risk Dataset")
print(f"Shape: {df_otif_risk.shape}")
print(f"\nTarget distribution (otif_risk):")
print("0 = Low Risk, 1 = Medium Risk, 2 = High Risk")
print(df_otif_risk['otif_risk'].value_counts().sort_index())
print(f"\nOrder type distribution:")
print(df_otif_risk['order_type'].value_counts())

display(df_otif_risk.head(10))

In [ ]:
spark.createDataFrame(df_otif_risk).write.mode("overwrite").saveAsTable("otif_risk")
print(f"✓ Saved to {CATALOG}.{SCHEMA}.otif_risk")

---
## 15. Verify All Tables

In [ ]:
# List all tables in the schema
print(f"Tables in {CATALOG}.{SCHEMA}:")
display(spark.sql(f"SHOW TABLES IN {CATALOG}.{SCHEMA}"))

In [ ]:
# Preview each table
tables = [
    ("demand_forecast", "Demand Planning - Time series forecasting"),
    ("price_elasticity", "Demand Planning - Price sensitivity"),
    ("promotion_lift", "Demand Planning - Promotion impact"),
    ("supplier_delay_risk", "Supply Planning - Predict delivery delays"),
    ("supplier_lead_time", "Supply Planning - Predict lead times"),
    ("material_shortage", "Supply Planning - Predict shortage risk"),
    ("labor_shortage", "Production Planning - Predict labor shortages"),
    ("yield_prediction", "Production Planning - Predict yield"),
    ("scrap_anomaly", "Production Planning - Detect anomalies"),
    ("transportation_lead_time", "Distribution Planning - Predict transit times"),
    ("otif_risk", "Distribution Planning - Predict OTIF risk"),
]

for table_name, description in tables:
    print(f"\n{'='*60}")
    print(f"{table_name}: {description}")
    print(f"{'='*60}")
    display(spark.table(table_name).limit(5))

## Summary

All datasets have been prepared and saved as Delta tables in `tabpfn_databricks.default`:

| Table | Task | Planning Process | Samples |
|-------|------|------------------|--------|
| `demand_forecast` | Time Series | Demand Planning | 1,800 |
| `price_elasticity` | Regression | Demand Planning | 3,000 |
| `promotion_lift` | Regression | Demand Planning | 2,500 |
| `supplier_delay_risk` | Binary Classification | Supply Planning | 2,000 |
| `supplier_lead_time` | Regression | Supply Planning | 2,000 |
| `material_shortage` | Multi-class Classification | Supply Planning | 1,500 |
| `labor_shortage` | Multi-class Classification | Production Planning | 1,500 |
| `yield_prediction` | Regression | Production Planning | 2,000 |
| `scrap_anomaly` | Anomaly Detection | Production Planning | 1,000 |
| `transportation_lead_time` | Regression | Distribution Planning | 2,000 |
| `otif_risk` | Multi-class Classification | Distribution Planning | 2,000 |

**Next steps:** Run the individual notebooks (01-04) to explore TabPFN capabilities using these prepared datasets.

### Planning Value Chain Coverage

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│ Demand Planning │───▶│ Supply Planning │───▶│ Production      │───▶│ Distribution    │
│                 │    │                 │    │ Planning        │    │ Planning        │
│ • Forecasting   │    │ • Supplier Risk │    │ • Labor Short.  │    │ • Transport LT  │
│ • Price Elast.  │    │ • Supplier LT   │    │ • Yield Pred.   │    │ • OTIF Risk     │
│ • Promo Lift    │    │ • Material      │    │ • Scrap Detect. │    │                 │
│                 │    │   Shortage      │    │                 │    │                 │
└─────────────────┘    └─────────────────┘    └─────────────────┘    └─────────────────┘
```